### Zadanie 1
Wykorzystaj funkcję finfo z biblioteki numpy aby określić precyzję i zakres różnych typów danych reprezentujących liczby zmienno-przecinkowe. Wyniki badania przedstaw w postaci zestawienia tabelarycznego.

In [ ]:
import numpy as np
from tabulate import tabulate

table = tabulate([['float64', np.finfo(np.float64).precision,
                   f'{np.finfo(np.float64).min} to {np.finfo(np.float64).max}'],
                  ['float32', np.finfo(np.float32).precision,
                   f'{np.finfo(np.float32).min} to {np.finfo(np.float32).max}'],
                  ['float16', np.finfo(np.float16).precision,
                   f'{np.finfo(np.float16).min} to {np.finfo(np.float16).max}']],
                 headers=['Type', 'Precision', 'Range'])
print(table)

### Zadanie 2 
Napisz funkcję, która dla dowolnej liczby zmiennoprzecinkowej w precyzji 64-bitowej (w standardzie IEEE 754) przedstawi w wersji dziesiętnej i binarnej składniki liczby: znak, wykładnik i mantysa. Można wykorzystać między innymi bibliotekę bitstring. Zbadaj jak reprezentowaną są poszczególne składniki tego formatu: zakres, sposób kodowania liczby etc.

In [ ]:
!pip install bitstring
!pip install tabulate
import bitstring as bs


def show_bin_and_dec(number):
    float64 = bs.BitArray(float=number, length=64)
    exponent = float64.bin[1:12]
    mantissa = float64.bin[12:]

    if float64[0] == 1:
        sign = '-'
    else:
        sign = '+'

    exponent_dec = 0
    pow = 10
    for i in exponent[0:]:
        if i == '1':
            exponent_dec += 2 ** pow
        pow -= 1

    pow = -1
    mantissa_dec = 0
    for i in mantissa[0:]:
        if i == '1':
            mantissa_dec += 2 ** pow
        pow -= 1

#     print('Floating point numbers (64-Bit):', '\n')
    print('number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)')
    print(number, '(dec) =', sign, f'(1 + {mantissa_dec}) x 2 ** ({exponent_dec} - 1023) = '
                                   f'{sign}{1 + mantissa_dec} x 2 ** {exponent_dec - 1023}', '\n')
    print("Sign Exponent Mantissa")
    print(float64.bin[0], exponent, mantissa)
    print(sign, exponent_dec, mantissa_dec, '\n\n')


show_bin_and_dec(1.2)
show_bin_and_dec(1.75)
show_bin_and_dec(1.25)


Liczby zmiennoprzecinkowe w precyzji 64-bitowej (wg standadru IEEE 754) są przechowywane w postaci 3 części:
1. Pierwszy bit od (od lewej) oznacza znak. Dla 0 mamy liczbę dodatnią a 1 oznacza -.
2. Następnie jest 11 bitów cechy. Za jej pomocą można "przesuwać" przecinek w liczbie. Jest to możliwe w obie strony (pozwala na reprezentację zarówno bardzo dużych jak i bardzo małych liczb - przez możliwość zwiększenia miejsca na część całkowitą lub ułamkową, zależnie od potrzeb). Pierwszy bit cechy oznacza, podobnie jak pierwszy bit całej liczby, znak. Liczba 'cecha - 1023' jest używana jako wykładnik potęgi 2 właśnie w celu przesuwania przecinka (stąd nazwa "floating point - reprezentacja zmiennoprzecikowa")
3. Na końcu zanjdują się 52 bity mantysy, kodującej właściwą liczbę. W celu normalizacji zawsze (w domyśle) dodawane jest do nich 1.

### Zadanie 3 
Wykorzystaj funkcję nextafter() z biblioteki numpy (lub inny sposób) i sprawdź rozdzielczość typu float (jakie mają wartości najbliższe sąsiadujące liczby float). Zaprezentuj wynik korzystając z funkcji z Zadania 2. 

In [ ]:
import numpy as np
# from zad2 import show_bin_and_dec


def next_float(x):
    print("Next (closest) value to {} is {}".format(x, np.nextafter(x, 1)))
    print(show_bin_and_dec(x))
    print(show_bin_and_dec(np.nextafter(x, 1)))


next_float(1.2)
next_float(1.75)
next_float(1.25)


### Zadanie 4
Porównać reprezentację bitową liczby 1/3 dla typów float o różnej precyzji: float16, float32, float64. Sprawdź co się dzieje gdy zmienna o mniejszej precyzji konwertowana jest do wyższej. 

In [ ]:
import numpy as np
from bitstring import BitArray


def float_dec2bin(number, bits=64):
    d_bin = BitArray(float=number, length=bits)
    sign = int(d_bin[0])
    if bits == 64:
        exp = d_bin[1:12].bin
        man = d_bin[12:].bin
    elif bits == 32:
        exp = d_bin[1:9].bin
        man = d_bin[9:].bin
    else:
        exp = d_bin[1:6].bin
        man = d_bin[6:].bin

    return sign, exp, man


print("float64:", float_dec2bin(np.float64(1/10)))
print("float32:", float_dec2bin(np.float32(1/3)))
print("float16:", float_dec2bin(np.float16(1/3)))

print("float16 -> float32:", float_dec2bin(np.float32(np.float16(1/3))))
print("float32 -> float64:", float_dec2bin(np.float64(np.float32(1/3))))
print("float16 -> float64:", float_dec2bin(np.float16(np.float32(1/3))))



Konwersja liczby z reprezentacji o mniejszej precyzji do większej nie ma większego sensu, gdyż liczba zapisana w mniejszej precyzji zawiera mniej informacji (jest mniej dokładna ze względu). Stąd też, kiedy konwertujemy ją do wyższej precyzji, nie zyskujemy nic ponieważ nasze dodatkowe (nowe) bity będą po prostu zerami. Z kolei konwersja w drugą stronę, tj. do mniejszych precyzji spowoduje utatę dokładności zapisanej liczby (czyli utratę części informacji o niej) - niejako "odetneimy" część bitów kodujących naszą liczbę.

### Zadanie 5
Sprawdź czym jest postać znormalizowana liczby typu float. Poniższy program generuje ciąg liczb. Wyświetl ich postać binarną i sprawdź czy są w postaci znormalizowanej (napisz stosowny kod, który to sprawdza). 

Liczba zmiennoprzecinkowa jest w postaci znormalizowanej gdy jej mantysa jest z przedzaiłu 0 a 1 (dla systemu dziesiętnego od 1 do 9 włącznie). Oznacza to, że cecha ktrórej używamy do "przesuwania" przecinka nie może być 0 w postaci znormalizowanej

In [ ]:
import numpy as np
from bitstring import BitArray


def to_binary(number):
    number_binary = BitArray(float=number, length=32)
    if number_binary[1:9].bin == '00000000':
        x = 'de'
    else:
        x = ''
    print(number)
    print(f'Is {x}normalised', int(number_binary[0]), number_binary[1:9].bin, number_binary[9:].bin, '\n')


a = np.float32(1.1)
for i in range(0, 149):
    a = a/np.float32(2.0)
    to_binary(a)


### Zadanie 6
Wyjaśnij dlaczego poniższy program nie działa prawidłowo. Sprawdź co się stanie gdy użyjemy typów o mniejszej precyzji. Zaproponuj inny sposób sumowania, który będzie dawał dokładne wyniki. 

In [ ]:
N = 100000000
C = 0.09531258654533566;

sum = 0.0 
for i in range(0, N):
    sum += C
    
print(sum)
print(N*C)

Powyższy błąd wynika z faktu, że liczby zmiennoprzecinkowe obarczone są pewnym błędem, spowodowanym skończoną liczbą bitów używanych do reprezentacji (potencjalnie nieskończonej liczby nieskończonych) wartości (czyli o skończonej precyzji). Dlatego też, sumując odpowiednio dużo razy wyżej podaną liczbę, za każdym razem zwiększamy nasz błąd. Wpewnym momencie wartość powyższej sumy może zacząć również przekraczać zakres wartości typu używanego do jej przechowywania. Poniżej prezentuję zestawienie dla sumowania "wprost" w 3 różnych stopniach precyzji oraz sumowanie z użyciem Algorytmu Sumacyjnego Kahana. Algorytm ten został stworzony właśnie w celu zminimalizowania błędów powodowanych przez sumowanie liczb zmiennoprzecinkowych o skończonej precyzji.

In [ ]:
import numpy as np
from tabulate import tabulate


def kahan_algorithm(x, n):
    sum = 0.0
    eps = 0.0
    for i in range(0, n):
        x_without_eps = x - eps
        tmp_sum = sum + x_without_eps
        eps = (tmp_sum - sum) - x_without_eps
        sum = tmp_sum
    return sum


def add_n_times(x, n, sum):
    for i in range(0, n):
        sum += x
    return sum


table = [["float16", add_n_times(np.float16(0.09531258654533566), 100000000, np.float16(0))],
        ["float32", add_n_times(np.float32(0.09531258654533566), 100000000, np.float32(0))],
        ["float64", add_n_times(np.float64(0.09531258654533566), 100000000, np.float64(0))],
        ["kahan algorithm", kahan_algorithm(0.09531258654533566, 100000000)],
        ["Correct value:", 0.09531258654533566*100000000]]

print(tabulate(table, headers=['Type', 'Value'], floatfmt=".10f"))